In [12]:
import requests
import psycopg2
import time
from bs4 import BeautifulSoup

In [13]:
conn = psycopg2.connect("dbname='movie_recommender' user='postgres' host='localhost' port=5432 password=''")
conn.autocommit = True

In [14]:
def updateRecord(record_to_insert):
    postgres_insert_query = "UPDATE idmapper set url='%s' where id=%d"%record_to_insert
    cur = conn.cursor()
    cur.execute(postgres_insert_query)
    conn.commit()


In [15]:
def hasEmptyUrlRecords():
    sql = "select count(*) from idmapper where url is null;"
    cur = conn.cursor()
    cur.execute(sql)
    record = None
    for row in cur:
        record = row
    return int(record[0])

In [16]:
def fetchWhereEmptyUrl(pid):
    sql = "select id, newid from idmapper where url is null and id=%d limit 1;"%(pid)
    cur = conn.cursor()
    cur.execute(sql)
    record = None
    for row in cur:
        record = row
    if not record: return (1, '')
    return record

In [17]:
def fetchImageUrl(movie_id):
    url = "https://www.imdb.com/title/%s/"%(movie_id)
    resp = requests.get(url)
    if resp.status_code >= 400:
        return ''
    img_url = ''
    try:
        bs = BeautifulSoup(resp.text)
        poster_div = bs.findAll("div", {"class": "poster"})[0]
        poster_img = poster_div.findAll('img')[0]
        img_url = poster_img['src']
    except:
        pass
    return img_url

In [19]:
for counter in [15157, 15158, 15159, 15160, 15161, 15162, 15163, 15164, 15165]:
    print "Fetching for ", counter
    rid, imdb_id = fetchWhereEmptyUrl(counter)
    print "Mapper info : ", rid, imdb_id, '\n'
    if imdb_id != '':
        img_url = fetchImageUrl(imdb_id)
        time.sleep(2)
        if img_url != '':
            updateRecord((img_url, rid))

Fetching for  15157
Mapper info :  15157 tt3305316 

Fetching for  15158
Mapper info :  15158 tt3816458 

Fetching for  15159
Mapper info :  15159 tt1645916 

Fetching for  15160
Mapper info :  15160 tt1595366 

Fetching for  15161
Mapper info :  15161 tt0239450 

Fetching for  15162
Mapper info :  15162 tt0465925 

Fetching for  15163
Mapper info :  15163 tt0248409 

Fetching for  15164
Mapper info :  15164 tt0289477 

Fetching for  15165
Mapper info :  15165 tt0128332 



In [10]:
# 8808 tt0464029 

In [11]:
conn.close()